### 크롤링 (차트, 장르, 플레이리스트)

- 인기차트

In [ ]:
def get_songs(i,j,k,df_together): # i:연대, j:연도, k:장르/스타일

    driver = webdriver.Chrome(options = chrome_options)
    driver.get('https://www.melon.com/chart/index.htm')
    time.sleep(3)

  ## -------[특정 연도 불러오기]----------------------------------------------------

    # 차트파인더 클릭
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, '//*[@id="gnb_menu"]/ul[1]/li[1]/div/div/button/span'))).click()

    # 연도차트 클릭
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, '//*[@id="d_chart_search"]/div/h4[3]/a'))).click()

    # 연대선택
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, f'//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[{i}]/span/label'))).click()

    # 연도선택
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, f'//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[{j}]/span/label'))).click()

    # 장르/스타일 선택
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, f'//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[{k}]/span/label'))).click()

    # 검색 버튼
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, '//*[@id="d_srch_form"]/div[2]/button/span/span'))).click()

    # 페이지 로드 기다리기
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((
            By.XPATH, '//*[@id="lst50"]/td[4]/div/div/div[1]/span/strong/a')))

    year = driver.find_element(By.XPATH, '//*[@id="serch_cnt"]/span[1]').text

  ## -------[노래 특성 가져오기]----------------------------------------------------

    # 리스트 리셋
    song_numbers = []
    titles = []
    artists = []
    lyrics = []
    whens = []
    likes = []
    genres = []

    # 각 노래의 고유한 번호 찾고 모으기
    ls50 = driver.find_elements(By.XPATH, '//*[@id="lst50"]/td[5]/div/button')
    ls100 = driver.find_elements(By.XPATH, '//*[@id="lst100"]/td[5]/div/button')

    for i in ls50:
      song_numbers.append(i.get_attribute('data-song-no'))
    for i in ls100:
      song_numbers.append(i.get_attribute('data-song-no'))

    # 노래 번호로 그 노래 상세 페이지 들어가기
    for number in song_numbers:
      driver = webdriver.Chrome(options = chrome_options) ####################
      driver.get(f'https://www.melon.com/song/detail.htm?songId={number}') # 새로운 페이지 오픈
      WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'song_name')))

      title = driver.find_element(By.CLASS_NAME, 'song_name').text
      artist = driver.find_element(By.CLASS_NAME, 'artist').text

      # 가사 없는 경우 np.nan으로 처리 (find_elements로 진행)
      lyric_element = driver.find_elements(By.CLASS_NAME, 'lyric')
      lyric =  lyric_element[0].text if lyric_element else np.nan

      when = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[2]').text
      like = driver.find_element(By.XPATH, '//*[@id="d_like_count"]').text
      genre = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[3]').text

      titles.append(title)
      artists.append(artist)
      lyrics.append(lyric)
      whens.append(when)
      likes.append(like)
      genres.append(genre)

    year_rep = np.repeat(year,len(titles))

    df = pd.DataFrame({'연도':year_rep, '제목':titles,'가수':artists, '가사':lyrics,
                      '발매년도':whens, '장르':genres, '좋아요 수':likes})

    df_together = pd.concat([df_together, df], ignore_index = True)
    return df_together

- 장르

In [ ]:
def get_songs_kor(i,j,k): # i:한국/해외, j:장르, k:스테디셀러

    driver = webdriver.Chrome(options = chrome_options)
    driver.get('https://www.melon.com/genre/song_list.htm?gnrCode=GN0100')
    time.sleep(5)

  ## -------[한국/해외 - 장르 - 스테디셀러 선택]----------------------------------------------------

    # 한국(1)/해외(2) 선택
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, f'//*[@id="gnb_menu"]/ul[1]/li[3]/div/ul/li[{i}]/a'))).click()

    # 장르 클릭
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, f'//*[@id="conts"]/div[2]/ul/li[{j}]/a'))).click()

    # 스테디셀러 클릭
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, f'//*[@id="conts"]/div[{k}]/ul/li[3]/a'))).click()

    # 페이지 로드 기다리기
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((
            By.XPATH, '//*[@id="frm"]/div/div/button[1]')))

    # 어떤 장르인지 저장
    genre = driver.find_element(By.XPATH, f'//*[@id="conts"]/div[2]/ul/li[{j}]/a/span').text

  ## -------[노래 특성 가져오기]----------------------------------------------------

    # 총 페이지 수
    pages = len(driver.find_element(By.XPATH, '//*[@id="pageObjNavgation"]/div/span').text)

    song_num = []

    for page in range(1, pages+1):

      # 페이지 잘 넘어가는지 체크하기 위함
      # print(driver.find_element(By.XPATH, '//*[@id="frm"]/div/table/tbody/tr[1]/td[2]/div/span[1]').text)

      if page < pages:

        for song in range(1,51):

          link = driver.find_element(By.XPATH, f'//*[@id="frm"]/div/table/tbody/tr[{song}]/td[7]/div/button')

          song_num.append(link.get_attribute('data-song-no'))

      else:

        for song in range(1,len(driver.find_elements(By.CLASS_NAME, 'rank'))):

          link = driver.find_element(By.XPATH, f'//*[@id="frm"]/div/table/tbody/tr[{song}]/td[7]/div/button')

          song_num.append(link.get_attribute('data-song-no'))

      driver.execute_script(f"pageObj.sendPage({page*50 + 1});")

      WebDriverWait(driver,10).until(EC.presence_of_element_located((
          By.XPATH, '//*[@id="frm"]/div/table/tbody/tr[1]/td[2]/div/span[1]')))

      time.sleep(6)

    print(f"총 {len(song_num)}곡의 노래를 가져옵니다!")
    # print(song_num)

      # 노래 번호 가져오기 완료 !

    titles = []
    artists = []
    lyrics = []
    whens = []
    likes = []

    for index in range(len(song_num)):

      number = song_num[index]

      # driver = webdriver.Chrome(options = chrome_options)
      driver.get(f'https://www.melon.com/song/detail.htm?songId={number}') # 새로운 페이지 오픈

      WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'song_name')))

      title = driver.find_element(By.CLASS_NAME, 'song_name').text
      artist = driver.find_element(By.CLASS_NAME, 'artist').text

      # 가사 없는 경우 np.nan으로 처리
      lyric_element = driver.find_elements(By.CLASS_NAME, 'lyric')
      lyric =  lyric_element[0].text if lyric_element else np.nan

      when = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[2]').text
      like = driver.find_element(By.XPATH, '//*[@id="d_like_count"]').text

      titles.append(title)
      artists.append(artist)
      lyrics.append(lyric)
      whens.append(when)
      likes.append(like)

      if index != 0 and index % 50 == 0:
        print("한 페이지 완료!")

      if index == len(song_num) - 1:
        print("노래 가져오기 끝!")

    genre_rep = np.repeat(genre, len(titles))

    df = pd.DataFrame({'장르':genre_rep, '제목':titles, '가수':artists, '가사':lyrics,
                      '발매년도':whens, '좋아요 수':likes})

    genre_clean = genre.replace('/', '-')

    df.to_pickle(f'googledrive/path/한국_{genre_clean}.pkl')

    return df

- 플레이리스트


In [ ]:
driver = webdriver.Chrome(options = chrome_options)

dj = 'boba' #####################################
# hashtags = []

for i in range(11,21):

  # 메인 페이지 불러오기
  driver.get('https://www.melon.com/mymusic/dj/mymusicdjplaylist_list.htm?memberKey=50700055') #####################################
  time.sleep(3)

  # playlist 고유 번호 가져오기 ----------------------------------
  link = driver.find_element(By.XPATH, f'//*[@id="pageList"]/div/div/table/tbody/tr[{i}]/td[4]/div/button') # i 번째 playlist 링크
  playlist = link.get_attribute('data-djcol-no') # playlist 고유 번호

  # playlist 안으로 이동 ----------------------------------
  driver.get(f'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq={playlist}')
  time.sleep(3)
  WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH,'//*[@id="frm"]/div/div/button[1]')))

  # 해시태그 가져오기
  hashtag_elements = driver.find_elements(By.CSS_SELECTOR, '.tag_list.type03 .tag_item')
  tags = [hashtag.text for hashtag in hashtag_elements]
  # hashtags.append(tags)

  # DJ 이름
  # dj = driver.find_element(By.XPATH, '//*[@id="conts"]/div[2]/div/div[2]/div[1]/div[2]/div/a').text

  # playlist 속 노래 개수
  songs = len(driver.find_elements(By.CLASS_NAME, 'rank')) # 총 노래 수 + 1

  titles = []
  artists = []
  lyrics = []
  whens = []
  likes = []

  # 각 노래마다 새로 창 열어가면서 진행 ----------------------------------
  for index in range(1,songs):

    driver.get(f'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq={playlist}')
    time.sleep(3)

    song_link = driver.find_element(By.XPATH, f'//*[@id="frm"]/div/table/tbody/tr[{index}]/td[7]/div/button')
    number = song_link.get_attribute('data-song-no') # 노래 고유 번호 가져오기

    driver.get(f'https://www.melon.com/song/detail.htm?songId={number}') # 상세 페이지 오픈
    time.sleep(3)

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'song_name')))

    title = driver.find_element(By.CLASS_NAME, 'song_name').text
    artist = driver.find_element(By.CLASS_NAME, 'artist').text

    # 가사 없는 경우 np.nan으로 처리
    lyric_element = driver.find_elements(By.CLASS_NAME, 'lyric')
    lyric =  lyric_element[0].text if lyric_element else np.nan

    when = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[2]').text
    like = driver.find_element(By.XPATH, '//*[@id="d_like_count"]').text
    genre = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[3]').text

    titles.append(title)
    artists.append(artist)
    lyrics.append(lyric)
    whens.append(when)
    likes.append(like)

    dj_rep = np.repeat(dj + '_' + str(i),len(titles))
  ## tag_rep = [tags for _ in range(len(titles))]

  df = pd.DataFrame({'플리':dj_rep, '제목':titles, '가수':artists, '가사':lyrics,
                      '장르':genre, '발매일':whens, '좋아요':likes})

  df['가사'] = df['가사'].str.replace('\n', ' ', regex=False)

  df.to_pickle(f'googledrive/path/{dj}/{dj}_{i}.pkl')

  print(f"{i} 번째 : ", tags, playlist)

  # print(f'{dj}의 {i}번째 playlist 저장 완료했습니다 :)')